In [1]:
!pip install datasets -q
!pip install transformers -q
!pip install jiwer -q

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
distributed 2024.9.0 requires dask==2024.9.0, but you have dask 2024.11.2 which is incompatible.
kfp 2.5.0 requires google-cloud-storage<3,>=2.2.1, but you have google-cloud-storage 1.44.0 which is incompatible.
libpysal 4.9.2 requires packaging>=22, but you have packaging 21.3 which is incompatible.
libpysal 4.9.2 requires shapely>=2.0.1, but you have shapely 1.8.5.post1 which is incompatible.
rapids-dask-dependency 24.10.0a0 requires dask==2024.9.0, but you have dask 2024.11.2 which is incompatible.
rapids-dask-dependency 24.10.0a0 requires dask-expr==1.1.14, but you have dask-expr 1.1.19 which is incompatible.


In [2]:
from huggingface_hub import login
login(token="") # your secret token here


In [3]:
from datasets import load_dataset

# Load the dataset
dataset = load_dataset("mozilla-foundation/common_voice_17_0", "en", split="train", streaming=True, trust_remote_code=True)
valid = load_dataset("mozilla-foundation/common_voice_17_0", "en", split="validation", streaming=True, trust_remote_code=True)
# test = load_dataset("mozilla-foundation/common_voice_17_0", "en", split="test", streaming=True, trust_remote_code=True)



README.md:   0%|          | 0.00/12.7k [00:00<?, ?B/s]

common_voice_17_0.py:   0%|          | 0.00/8.19k [00:00<?, ?B/s]

languages.py:   0%|          | 0.00/3.92k [00:00<?, ?B/s]

release_stats.py:   0%|          | 0.00/132k [00:00<?, ?B/s]

In [4]:
dataset=dataset.remove_columns(['client_id', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment', 'variant'])
valid=valid.remove_columns(['client_id', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment', 'variant'])
# test=test.remove_columns(['client_id', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment', 'variant'])

In [5]:
import pandas as pd
import numpy as np

In [6]:
data = []
num_examples_to_load = 2200

for i, example in enumerate(dataset):
    # Calculate the duration
    duration = example['audio']['array'].shape[0] / example['audio']['sampling_rate']
    length = len(example.get('sentence', ''))

    if (1 <= duration <= 5) and length >= 3:
        data.append({
            'path': example['path'],
            'audio_array': example['audio']['array'],
            'sampling_rate': example['audio']['sampling_rate'],
            'duration': duration,
            'text': example.get('sentence', ''),
        })

    if len(data) >= num_examples_to_load:
        break

# Convert the list to a DataFrame
df_10 = pd.DataFrame(data)

Reading metadata...: 1101170it [00:17, 61653.76it/s]


In [7]:
# data_test = []
# num_examples_to_load = 200

# for i, example in enumerate(test):
#     # Calculate the duration
#     duration = example['audio']['array'].shape[0] / example['audio']['sampling_rate']
#     length = len(example.get('sentence', ''))

#     if (1 <= duration <= 5) and length >= 3:
#         data_test.append({
#             'path': example['path'],
#             'audio_array': example['audio']['array'],
#             'sampling_rate': example['audio']['sampling_rate'],
#             'duration': duration,
#             'text': example.get('sentence', ''),
#         })

#     if len(data_test) >= num_examples_to_load:
#         break

# # Convert the list to a dataframe
# test = pd.DataFrame(data_test)

In [8]:
data_valid = []
num_examples_to_load = 400

for i, example in enumerate(valid):
    # Calculate the duration
    duration = example['audio']['array'].shape[0] / example['audio']['sampling_rate']
    length = len(example.get('sentence', ''))

    if (1 <= duration <= 5) and length >= 3:
        data_valid.append({
            'path': example['path'],
            'audio_array': example['audio']['array'],
            'sampling_rate': example['audio']['sampling_rate'],
            'duration': duration,
            'text': example.get('sentence', ''),
        })

    if len(data_valid) >= num_examples_to_load:
        break

# Convert the list to a dataframe
valid = pd.DataFrame(data_valid)

Reading metadata...: 16393it [00:00, 32658.49it/s]


In [9]:
from datasets import Dataset
dataset = Dataset.from_pandas(df_10)
valid = Dataset.from_pandas(valid)
# test = Dataset.from_pandas(test)

In [10]:
# # Save datasets
# dataset.save_to_disk('dataset_direc')
# test.save_to_disk('test_direc')

# # Compress the directories for download
# !zip -r dataset_direc.zip dataset_direc
# !zip -r test_direc.zip test_direc


In [11]:
import torch
import torchaudio

def resample_audio(batch):
    resampler = torchaudio.transforms.Resample(orig_freq=48000, new_freq=16000)
    batch['audio_array'] = [resampler(torch.tensor(audio, dtype=torch.float32)).numpy() if len(audio) > 0 else np.array([], dtype=np.float32) for audio in batch['audio_array']]  # Directly create tensor from audio
    batch['sampling_rate'] = [16000] * len(batch['audio_array'])
    return batch

# Apply the resampling function
dataset = dataset.map(resample_audio, batched=True)
valid = valid.map(resample_audio, batched=True)
# test = test.map(resample_audio, batched=True)

Map:   0%|          | 0/2200 [00:00<?, ? examples/s]

Map:   0%|          | 0/400 [00:00<?, ? examples/s]

In [12]:
import re
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"]'

def remove_special_characters(batch):
    batch["text"] = re.sub(chars_to_ignore_regex, '', batch["text"]).lower() + " "
    return batch

In [13]:
dataset = dataset.map(remove_special_characters)
valid = valid.map(remove_special_characters)
# test = test.map(remove_special_characters)

Map:   0%|          | 0/2200 [00:00<?, ? examples/s]

Map:   0%|          | 0/400 [00:00<?, ? examples/s]

In [14]:
def extract_all_chars(batch):
  all_text = " ".join(batch["text"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

In [15]:
import datasets
vocabs_dataset = dataset.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=["path", "audio_array", "sampling_rate","duration", "text"])
vocabs_test = valid.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=["path", "audio_array", "sampling_rate","duration", "text"])
vocabs = datasets.concatenate_datasets([vocabs_dataset, vocabs_test])

Map:   0%|          | 0/2200 [00:00<?, ? examples/s]

Map:   0%|          | 0/400 [00:00<?, ? examples/s]

In [16]:
vocabs

Dataset({
    features: ['vocab', 'all_text'],
    num_rows: 2
})

In [17]:
vocab_list = list(set(vocabs["vocab"][0]))

vocab_dict = {v: k for k, v in enumerate(vocab_list)}
vocab_dict

{'u': 0,
 '’': 1,
 'v': 2,
 'i': 3,
 'w': 4,
 'k': 5,
 'a': 6,
 'm': 7,
 'h': 8,
 'q': 9,
 '‘': 10,
 ')': 11,
 'z': 12,
 'j': 13,
 '(': 14,
 's': 15,
 'g': 16,
 '“': 17,
 'd': 18,
 'n': 19,
 'o': 20,
 '”': 21,
 'c': 22,
 '—': 23,
 'â': 24,
 'p': 25,
 'r': 26,
 'e': 27,
 'é': 28,
 'í': 29,
 'f': 30,
 'l': 31,
 'y': 32,
 't': 33,
 'b': 34,
 'ê': 35,
 "'": 36,
 'x': 37,
 ' ': 38}

In [18]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
print(len(vocab_dict))

41


In [19]:
import json
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [20]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [21]:
from transformers import SeamlessM4TFeatureExtractor

feature_extractor = SeamlessM4TFeatureExtractor.from_pretrained("facebook/w2v-bert-2.0")


preprocessor_config.json:   0%|          | 0.00/275 [00:00<?, ?B/s]

In [22]:
from transformers import Wav2Vec2BertProcessor

processor = Wav2Vec2BertProcessor(feature_extractor=feature_extractor, tokenizer=tokenizer)


In [23]:
import random

rand_int = random.randint(0, len(dataset)-1)

print("Target text:", dataset[rand_int]["text"])
print("Input array shape:", np.asarray(dataset[rand_int]["audio_array"]).shape)
print("Sampling rate:", dataset[rand_int]["sampling_rate"])

Target text: however the tbones finished third in both halves of the season 
Input array shape: (74880,)
Sampling rate: 16000


In [24]:
import librosa
def prepare_dataset(batch):
    audio = batch
    batch["input_features"] = processor(audio["audio_array"], sampling_rate=audio["sampling_rate"]).input_features[0]
    batch["input_length"] = len(batch["input_features"])

    batch["labels"] = processor(text=batch["text"]).input_ids
    return batch


In [25]:
dataset

Dataset({
    features: ['path', 'audio_array', 'sampling_rate', 'duration', 'text'],
    num_rows: 2200
})

In [26]:
dataset = dataset.map(prepare_dataset, remove_columns=["path" , "audio_array", "duration", "sampling_rate", "text"], num_proc=1)
valid = valid.map(prepare_dataset, remove_columns=["path" , "audio_array", "duration", "sampling_rate", "text"], num_proc=1)
# test = test.map(prepare_dataset, remove_columns=["path" , "audio_array", "duration", "sampling_rate", "text"], num_proc=1)

Map:   0%|          | 0/2200 [00:00<?, ? examples/s]

Map:   0%|          | 0/400 [00:00<?, ? examples/s]

In [27]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:

    processor: Wav2Vec2BertProcessor
    padding: Union[bool, str] = True

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            return_tensors="pt",
        )

        labels_batch = self.processor.pad(
            labels=label_features,
            padding=self.padding,
            return_tensors="pt",
        )
        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch



In [28]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [29]:
!pip install evaluate -q
import evaluate

# Load the WER metric
wer_metric = evaluate.load("wer")

In [30]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}


In [31]:
from transformers import Wav2Vec2BertForCTC

model = Wav2Vec2BertForCTC.from_pretrained(
    "facebook/w2v-bert-2.0",
    attention_dropout=0.1,
    hidden_dropout=0.1,
    feat_proj_dropout=0.1,
    mask_time_prob=0.05,
    layerdrop=0.01,
    ctc_loss_reduction="mean",
    add_adapter=True,
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer),
)


config.json:   0%|          | 0.00/1.87k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.32G [00:00<?, ?B/s]

Some weights of Wav2Vec2BertForCTC were not initialized from the model checkpoint at facebook/w2v-bert-2.0 and are newly initialized: ['adapter.layers.0.ffn.intermediate_dense.bias', 'adapter.layers.0.ffn.intermediate_dense.weight', 'adapter.layers.0.ffn.output_dense.bias', 'adapter.layers.0.ffn.output_dense.weight', 'adapter.layers.0.ffn_layer_norm.bias', 'adapter.layers.0.ffn_layer_norm.weight', 'adapter.layers.0.residual_conv.bias', 'adapter.layers.0.residual_conv.weight', 'adapter.layers.0.residual_layer_norm.bias', 'adapter.layers.0.residual_layer_norm.weight', 'adapter.layers.0.self_attn.linear_k.bias', 'adapter.layers.0.self_attn.linear_k.weight', 'adapter.layers.0.self_attn.linear_out.bias', 'adapter.layers.0.self_attn.linear_out.weight', 'adapter.layers.0.self_attn.linear_q.bias', 'adapter.layers.0.self_attn.linear_q.weight', 'adapter.layers.0.self_attn.linear_v.bias', 'adapter.layers.0.self_attn.linear_v.weight', 'adapter.layers.0.self_attn_conv.bias', 'adapter.layers.0.self_

In [32]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir=r"FinalResult",
  group_by_length=True,
  per_device_train_batch_size=16,
  per_device_eval_batch_size=16,
  eval_strategy="steps",
  num_train_epochs=30,
  fp16=True,
  gradient_checkpointing=True,
  save_steps=500,
  eval_steps=500,
  logging_steps=500,
  learning_rate=3e-5,
  weight_decay=0.001,
  warmup_steps=500,
  save_total_limit=1,
  load_best_model_at_end=True, 
  metric_for_best_model="wer",  
  greater_is_better=False
)

In [33]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=dataset,
    eval_dataset=valid,
    processing_class=processor.feature_extractor,
)

In [34]:
import wandb
wandb.login(key="444f2c6a6660f596352ee3f66f87d31fbc15002c")
trainer.train()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: royalstha5567 (royalstha5567-lalitpur-engineering-college). Use `wandb login --relogin` to force relogin


/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


Step,Training Loss,Validation Loss,Wer
500,2.102500,0.591349,0.542215
1000,0.593100,0.542602,0.465052
1500,0.348600,0.543238,0.394118
2000,0.288100,0.637396,0.387543
2500,0.156400,0.732169,0.374394
3000,0.112500,0.625905,0.371626
3500,0.082900,0.738155,0.356747
4000,0.105500,0.787507,0.355709


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
Trainer.toke

TrainOutput(global_step=4140, training_loss=0.4600703455975666, metrics={'train_runtime': 13108.5955, 'train_samples_per_second': 5.035, 'train_steps_per_second': 0.316, 'total_flos': 7.959885040794939e+18, 'train_loss': 0.4600703455975666, 'epoch': 30.0})

In [35]:
test_results = trainer.evaluate(valid)
print("Test Loss:", test_results["eval_loss"])
print("Test WER:", test_results["eval_wer"])

Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.


Test Loss: 0.7875118851661682
Test WER: 0.35570934256055364


In [39]:
valid

Dataset({
    features: ['input_features', 'input_length', 'labels'],
    num_rows: 400
})

In [40]:
def map_to_result(batch):
  with torch.no_grad():
    input_values = torch.tensor(batch["input_features"], device="cuda").unsqueeze(0)
    logits = model(input_values).logits

  pred_ids = torch.argmax(logits, dim=-1)
  batch["pred_str"] = processor.batch_decode(pred_ids)[0]
  batch["text"] = processor.decode(batch["labels"], group_tokens=False)
  
  return batch

results = valid.map(map_to_result, remove_columns=valid.column_names)


Map:   0%|          | 0/400 [00:00<?, ? examples/s]

In [37]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))


In [41]:
results

Dataset({
    features: ['pred_str', 'text'],
    num_rows: 400
})

In [42]:
show_random_elements(results)

,pred_str,text
0,it seemed so close to her,it seemed so close to her
1,there is no office no salaries and no actual buss,there is no office no salaries and no actual boss
2,is your chint really frick,is your child really fit
3,what is the benefit of the method in terms of identification accuracy,what is the benefit of the method in terms of identification accuracy
4,and that's my fare well to the newspaper game,and that's my farewell to the newspaper game
5,he died at crasweek,he died at chiswick
6,the show was a flop from the very start,the show was a flop from the very start
7,they were out of distance of sfeers,they were out of distance of spears
8,the college is located on airoad chchirt mumbai,the college is located on 'a' road churchgate mumbai
9,they said nothing else,they said nothing else


In [43]:
SAVE_DIR = '/kaggle/working/fine_tuned_model/'

# Save the fine-tuned model and processor
model.save_pretrained(SAVE_DIR)
processor.save_pretrained(SAVE_DIR)

[]